SE IMPORTAN LAS LIBRERÍAS PERTINENTES

In [67]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler,MaxAbsScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

SE CARGAN LOS CSV DE TEST Y TRAIN

In [68]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

PASAMOS A NUMERICO Y PONEMOS A NAN TODOS LOS ? Y LOS NÚMEROS NEGATIVOS

In [69]:
train=train.apply(pd.to_numeric, errors='coerce')
train=train.replace({"?":np.nan})
train[train < 0] = np.nan

test=test.apply(pd.to_numeric, errors='coerce')
test=test.replace({"?":np.nan})
test[test < 0] = np.nan

LAS COLUMNAS DONDE HAY 0 Y ES UN VALOR IRREAL, TAMBIÉN LAS PONEMOS A NULO

In [70]:
cols_error=['trestbps', 'chol','oldpeak']
train[cols_error] = train[cols_error].replace({0: np.nan})

cols_error=['trestbps', 'chol','oldpeak']
test[cols_error] = test[cols_error].replace({0: np.nan})

In [71]:
train.to_csv('train1.csv')

IMPUTAMOS LOS NULOS CON LA MEDIANA

In [72]:
for col in test.columns:
    if train[col].isnull().any():
        median_val = train[col].median()
        train[col] = train[col].fillna(median_val)

    if test[col].isnull().any():
        median_val = test[col].median()
        test[col] = test[col].fillna(median_val)

In [73]:
def medical_feature_engineering(df_raw):
    """
    Transforma datos crudos en un perfil de riesgo cardiológico completo.
    Nivel Avanzado: Hemodinámica, Metabolismo y Electrofisiología.
    """
    df = df_raw.copy()

    # --- 1. LIMPIEZA INICIAL (Sanity Check) ---
    # Convertimos a NaN valores imposibles para que MICE los arregle después
    for col in ['trestbps', 'chol']:
        df[col] = pd.to_numeric(df[col], errors='coerce')
        df[col] = df[col].apply(lambda x: np.nan if x <= 10 else x) # Umbral 10 por seguridad

    # --- 2. SUBSISTEMA HEMODINÁMICO (Mecánica de fluidos) ---
    
    # Rate-Pressure Product (Consumo de O2)
    df['RPP'] = df['thalach'] * df['trestbps']
    
    # Presión Arterial Media Estimada (MAP Proxy)
    # Estima la presión de perfusión constante.
    df['MAP_Proxy'] = df['trestbps'] * 0.70 

    # --- 3. SUBSISTEMA CRONOTRÓPICO (Respuesta del corazón) ---
    
    # Max Heart Rate Achieved vs Theoretical (Incompetencia Cronotrópica)
    # Cuanto más cerca de 1, mejor. Si es bajo, el corazón falla al acelerar.
    # Evitamos división por cero en ancianos extremos (aunque 220-age rara vez es 0)
    df['HR_Reserve_Ratio'] = df['thalach'] / (220 - df['age'])

    # --- 4. SUBSISTEMA METABÓLICO (Química sanguínea) ---
    
    # Carga Metabólica (Interacción Diabetes-Colesterol)
    # Si fbs es 1 (diabético), el colesterol se penaliza el doble.
    df['Metabolic_Risk'] = df['chol'] * (1 + df['fbs'])

    # --- 5. SUBSISTEMA ELÉCTRICO/ISQUÉMICO (Daño estructural) ---
    
    # Riesgo Eléctrico (Ya lo teníamos, es muy bueno)
    df['ECG_Risk'] = df['oldpeak'] * df['slope']

    
    # Índice de Angina Combinada
    # cp: 1=typical, 2=atypical, 3=non-anginal, 4=asymptomatic
    # Transformamos cp para que 1 sea el más grave y 4 el menos (invertimos lógica original si es necesario)
    # Asumiendo el dataset original: 4 suele ser asintomático (peligroso en silencioso) o no-cardiaco.
    # Vamos a usar la interacción directa Angina Ejercicio + Oldpeak
    df['Ischemia_Severity'] = df['exang'] * df['oldpeak']

    df['Silent_Ischemia'] = ((df['cp'] == 4) & (df['exang'] == 1)).astype(int)


    # --- 6. TRANSFORMACIONES MATEMÁTICAS (Normalización de distribución) ---
    # Logaritmos para variables con colas largas (Outliers naturales)
    df['log_chol'] = df['chol'].apply(lambda x: np.log1p(x) if x > 0 else np.nan)
    df['log_oldpeak'] = df['oldpeak'].apply(lambda x: np.log1p(x) if x >= 0 else np.nan)
    
    # RPP suele ser un número gigante (ej. 150*120 = 18000), aplicamos log para suavizarlo
    df['log_RPP'] = df['RPP'].apply(lambda x: np.log1p(x) if x > 0 else np.nan)

    return df

AGREGAMOS LAS NUEVAS COLUMNAS DEL FEATURE ENGINEERING

In [74]:
train=medical_feature_engineering(train)
test=medical_feature_engineering(test)

ELIMINAMOS LAS COLUMNAS QUE HEMOS DESCARTADO AL HACER EL EDA TANTO DE LOS NUEVOS DATOS COMO DE LOS VIEJOS

In [75]:
train=train.drop(columns=["chol","log_chol","trestbps","log_RPP","log_oldpeak","MAP_Proxy","thal","thalach"])

test=test.drop(columns=["chol","log_chol","trestbps","log_RPP","log_oldpeak","MAP_Proxy","thal","thalach"])


DIVIDIMOS EN X E y PARA DEFINIR CARACTERÍSTICAS Y ETIQUETA

In [76]:
X = train.drop(columns=["label"])
y = train["label"]

PARTIMOS EL SET DE TRAIN PARA PODER HACER LA VALIDACIÓN Y APLICAMOS LA REGRESION LOGISTICA

In [77]:
# ==========================
# 1. Train / Validation split
# ==========================
X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y  # mantener proporción de clases
)

# ==========================
# 2. Pipeline: scaler + Logistic Regression
# ==========================
logreg_model = Pipeline([
    ("scaler", StandardScaler()),
    ("logreg", LogisticRegression(
        max_iter=400,
        multi_class="auto",
        random_state=42,
        n_jobs=-1
    ))
])

# ==========================
# 3. Entrenar
# ==========================
logreg_model.fit(X_train, y_train)

# ==========================
# 4. Predicción y métricas
# ==========================
y_pred = logreg_model.predict(X_val)

f1_micro = f1_score(y_val, y_pred, average="micro")
f1_macro = f1_score(y_val, y_pred, average="macro")

print("F1-micro:", f1_micro)
print("F1-macro:", f1_macro)



c:\Users\sevil\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


F1-micro: 0.48299319727891155
F1-macro: 0.292902809906983


PREDECIMOS EL SET DE TEST Y OBTENEMOS LAS PREDICCIONES QUE NOS GUARDAMOS EN UN CSV Y MOSTRAMOS LOS PRIMEROS VALORES

In [78]:
# ==========================
# 1️⃣ Preparar test con mismas columnas que X
# ==========================
X_test = test[X.columns].fillna(0).astype(float)  # rellenar NaN con 0

# ==========================
# 2️⃣ Escalar el test y predecir
# ==========================
y_test_pred_logreg = logreg_model.predict(X_test)

# ==========================
# 3️⃣ Crear DataFrame con predicciones
# ==========================
df_predicciones_logreg = pd.DataFrame({
    "ID":X_test.index,
    "label": y_test_pred_logreg})

print(df_predicciones_logreg.head())

# ==========================
# 4️⃣ Guardar CSV con predicciones
# ==========================
df_predicciones_logreg.to_csv(
    'predicciones.csv',
)


   ID  label
0   0      4
1   1      0
2   2      0
3   3      2
4   4      0
